In [ ]:
%pip install tensorflow
%pip install keras
%pip install matplotlib
%pip install pillow
%pip install tqdm

In [3]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from PIL import Image, ImageFile
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [4]:
IMAGE_SIZE = [300, 400]
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False

In [5]:
x = Flatten()(vgg.output)
preds = Dense(2, activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=preds)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 300, 400, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 300, 400, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 300, 400, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 150, 200, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 150, 200, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 150, 200, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 75, 100, 128)      0     

In [6]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=['accuracy'],
)

In [5]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
TRAIN_DATA_PATH = "../../my-datasets/FloodNet Challenge @ EARTHVISION 2021 - Track 1/Train/Classification"
CATEGORIES = ["Flooded", "Non-Flooded"]
train_data_list = []

for category in CATEGORIES:
    path = os.path.join(TRAIN_DATA_PATH, category)
    class_num = CATEGORIES.index(category)
    for img_path in tqdm(os.listdir(path)):
        try:
            img = Image.open(os.path.join(path, img_path))
            img = np.array(img.resize((400, 300)))
            train_data_list.append([img, class_num])
        except Exception as e:
            print(e)

100%|██████████| 348/348 [01:09<00:00,  5.01it/s]

[Errno 21] Is a directory: '../../my-datasets/FloodNet Challenge @ EARTHVISION 2021 - Track 1/Train/Classification/Non-Flooded/.ipynb_checkpoints'


In [6]:
import random
random.shuffle(train_data_list)

X = []
y = []

for image, label in train_data_list:
    X.append(image)
    y.append(label)

X = np.array(X).reshape(-1, 300, 400, 3)
y = np.array(y)

#np.save("X_images", X, allow_pickle=True)
#np.save("y_labels", y, allow_pickle=True)

In [7]:
X = np.load("X_images.npy")
y = np.load("y_labels.npy")

model.fit(
    x=X,
    y=y,
    batch_size=8,
    epochs=3,
    verbose=1,
) 

Epoch 1/3
50/50 [==============================] - 15s 171ms/step - loss: 4.2575 - accuracy: 0.9146
Epoch 2/3
50/50 [==============================] - 5s 109ms/step - loss: 0.0148 - accuracy: 0.9975
Epoch 3/3
50/50 [==============================] - 5s 109ms/step - loss: 0.0233 - accuracy: 0.9975


In [8]:
model.evaluate(x=X, y=y, batch_size=16)

25/25 [==============================] - 16s 407ms/step - loss: 0.0000e+00 - accuracy: 1.0000


[0.0, 1.0]

In [20]:
preds = model.predict(x=X)
preds_ = np.argmax(preds, axis=1)

total = 0
correct = 0

for pred, truth in zip(preds_, y):
    if(pred == truth):
        correct += 1
    total += 1
    
print(f"Accuracy: {(correct/total)*100}%")

Accuracy: 100.0%
